## Введение

Датасет содержит данные о стационарных **предприятиях общественного питания**, расположенных в пределах установленных границ города **Москвы**. Данные обновляются ежеквартально и последний раз обновлялись 08.10.2024.

**Источник**: Портал открытых данных правительства Москвы. Ссылка на датасет: https://data.mos.ru/opendata/1903?pageSize=10&pageIndex=0&isDynamic=false&version=1&release=197

#### Что содержит датасет?
- `ID`
- `Name` - название заведения
- `IsNetObject` - является ли заведение сетевым
- `TypeObject` - тип заведения
- `AdmArea` - в каком административном округе находится
- `District` - в каком районе находится
- `Address` - полный адрес заведения
- `SeatsCount` - сколько посадочных мест есть в заведении
- `SocialPrivileges` - есть ли какие-то социальные льготы в заведении


## Загрузка даннных

In [1]:
# Откроем файл, прочитаем его как список строк 
with open('data/food_service.csv', encoding='utf-8-sig') as f:
    file = f.readlines()
# Теперь посмотрим как выглядят первые 5 строк
print (file[:4])

['ID;Name;IsNetObject;TypeObject;AdmArea;District;Address;SeatsCount;SocialPrivileges;Longitude_WGS84;Latitude_WGS84\n', '00151635;СМЕТАНА;нет;кафе;Северо-Восточный административный округ;Ярославский район;Российская Федерация, город Москва, внутригородская территория муниципальный округ Ярославский, улица Егора Абакумова, дом 9;44;нет;37.7145650004360390;55.8790015313033730\n', '000077874;Родник;нет;кафе;Центральный административный округ;Таганский район;город Москва, улица Талалихина, дом 2/1, корпус 1;35;нет;37.6733061300344000;55.7382386551547000\n', '000024309;Кафе «Академия»;нет;ресторан;Центральный административный округ;Таганский район;Российская Федерация, город Москва, внутригородская территория муниципальный округ Таганский, Абельмановская улица, дом 6;95;нет;37.6696475969381000;55.7355114718314000\n']


Мы видим, что первая строка это название парметров, которые характеризует заведение, а далее идут собственно заведения и информация о них.


## Создание словаря
Теперь создадим словарь, в котором будет содержаться информация о заведениях.
 
Ключом словаря будет являться название заведения. Но названия заведений могут повторяться, а находиться в разных местах, например, сетевые заведения.
 
Поэтому если название заведения уникально в файле, то ключ будет хранить список, в котором будет один словарь со следующими ключами: IsNetObject, TypeObject, AdmArea, District, Address, SeatsCount, SocialPrivileges, Longitude_WGS84, Latitude_WGS84.
 
Если же название заведения повторяется в файле, то ключ будет содержать список словарей, и каждый словарь будет представлять одно заведение с этим названием, но с различными параметрами (например, разные адреса).
 

In [40]:
# Создаем пустой словарь
food_service_dict = {}
    
# Открываем файл и читаем его по строкам
with open ('data/food_service.csv', encoding = 'utf-8-sig') as f:
        # Прочитаем первую строку, так там находятся заголовки
        headings = f.readline()
        # Теперь разделим их по знаку-разделителю ";" и удерем лишние пробелы вокруг слов на всякий случай
        headings = headings.strip().split(';')
        
        # Далее находим индексы каждого столбца-заголовка, чтобы понять из какого места нужно будет забирать данные в каждой строке
        name_index = headings.index('Name')
        net_index = headings.index('IsNetObject')
        type_index = headings.index('TypeObject')
        adm_index = headings.index('AdmArea')
        district_index = headings.index('District')
        address_index = headings.index('Address')
        seats_index = headings.index('SeatsCount')
        social_index = headings.index('SocialPrivileges')
        long_index = headings.index('Longitude_WGS84')
        lat_index = headings.index('Latitude_WGS84')
        
        # Далее для каждой строке в файле разбиваем её по знаку-разделителю
        for line in f:
            inf = line.strip().split(';')
            name = inf[name_index]
                
            # Теперь добавляем нужные элементы в словарь
            data = {'IsNetObject': inf[net_index],
                    'TypeObject': inf[type_index],
                    'AdmArea': inf[adm_index],
                    'District': inf[district_index],
                    'Address': inf[address_index],
                    'SeatsCount': inf[seats_index],
                    'SocialPrivileges': inf[social_index],
                    'Longitude_WGS84': inf[long_index],
                    'Latitude_WGS84': inf[lat_index]}
                
            # Проверяем, если заведение с таким названием уже есть, то добавляем его запись в список 
            if name in food_service_dict:
                    food_service_dict[name].append(data)
                # Если название еще не встречалось, то создаем новый список со словарем внутри
            else: food_service_dict[name] = [data]

Посмотрим как выглядит информация о кафе `Брусника`, например.


In [ ]:
print(food_service_dict['Брусника'])

Теперь, когда словарь готов можем немного проанализировать данные.

## Количество заведений и мест общественного питания
- `Заведением` будем называть точку общественного питания с уникальным названием.
- `Место` - это просто точка общественного питания. Места могут иметь одинаковое название, но различаться по остальным параметрам.
Для начала найдем количество заведений, которые имеют уникально название, т.е. заведения с одинаковым названием будем считать за одно.

In [3]:
number = len(food_service_dict)
print('Количество заведений с уникальным названием равно', number)

Количество заведений с уникальным названием равно 13695


Далее найдем в общем количество мест, которые содержаться в этом датасете вне зависимости от названия.

In [4]:
# Запускаем счетчик
count = 0
# Проходимся по всем ключам и добавляем длину их значений к счетчику
for places in food_service_dict:
    count += len(places)
print('Общее количество мест общественного питания равно', count)

Общее количество мест общественного питания равно 205505


## Количество мест по административным районам
Найдем сколько мест общественного питания находится в каждом административном округе и отсортируем их в порядке убывания.

In [31]:
# Создаем пустой словарь, в котором ключом будет адм. округ, а значением - количесвто мест
adm_count = {}
# Проходимся по всем записям в словаре и получаем названия адм. округов
for places in food_service_dict.values(): 
    for place in places:
        adm_area = place['AdmArea']
        
        # Если адм. округ уже есть в словаре, то увеличиваем количество, если нет, то начинаем считать
        if adm_area in adm_count: 
            adm_count[adm_area] += 1
        else: adm_count[adm_area] = 1 

adm_count_sorted = sorted(adm_count, key=adm_count.get, reverse=True)
for adm_area in adm_count_sorted:
    print(adm_area,':', adm_count[adm_area], 'мест(а)')

Центральный административный округ : 5670 мест(а)
Южный административный округ : 2488 мест(а)
Северный административный округ : 2032 мест(а)
Западный административный округ : 1995 мест(а)
Северо-Восточный административный округ : 1981 мест(а)
Восточный административный округ : 1713 мест(а)
Юго-Западный административный округ : 1648 мест(а)
Юго-Восточный административный округ : 1430 мест(а)
Северо-Западный административный округ : 1110 мест(а)
Новомосковский административный округ : 984 мест(а)
Зеленоградский административный округ : 341 мест(а)
Троицкий административный округ : 183 мест(а)


## Типы мест общественного питания
Найдем сколько мест общественного питания относится к каждому типу и отсортируем их в порядке уменьшения количества мест.

In [6]:
# Создаем пустой словарь, в котором ключом будет тип объекта, а значением - количесвто мест.
type_count = {} 
# Проходимся по всем записям в словаре и получаем названия типов
for places in food_service_dict.values(): 
    for place in places:
        type = place['TypeObject']
        
        # Если тип уже есть в словаре, то увеличиваем количество мест
        if type in type_count: 
            type_count[type] += 1
        # Если нет, то начинаем считать
        else: type_count[type] = 1 
            
# Отсортируем по уменьшению количества мест для каждого типа и выведем
type_count_sorted = sorted(type_count, key=type_count.get, reverse=True)
for type in type_count_sorted:
    print(type,':', type_count[type], 'мест(а)')

кафе : 7704 мест(а)
предприятие быстрого обслуживания : 3799 мест(а)
ресторан : 2524 мест(а)
кафетерий : 2227 мест(а)
столовая : 2121 мест(а)
бар : 1446 мест(а)
закусочная : 646 мест(а)
магазин (отдел кулинарии) : 463 мест(а)
буфет : 411 мест(а)
заготовочный цех : 156 мест(а)
ночной клуб (дискотека) : 78 мест(а)


## Поиск мест в зависимости от района Москвы


In [48]:
def places_by_district (food_service_dict, district):
    # Создаем cписок для хранения мест в заданном районе
    places_in_district = []
    # Проходим по всем ключам-названиям в словаре
    for name in food_service_dict:
        places = food_service_dict[name]   # Получаем список словарей для текущего названия
        for place in places:
            if place['District'] == district: # проверяем соответствует ли район
                places_in_district.append({   # если да, то добавляем новый словарь в список
                    'Name': name,
                    'Address': place['Address'],
                    'Type': place['TypeObject'], 
                    'SeatsCount': place['SeatsCount'],
                    'SocialPrivileges': place['SocialPrivileges']}) 
    return (places_in_district)


Например, можно найти места общественного питания в районе Марьино

In [47]:
district = 'район Марьино'
places_marino = places_by_district(food_service_dict, district)
print('Всего найдено мест:', len(places_marino))
for place in places_marino:
    print('Название:', place['Name'])
    print('Тип:', place['Type'])  
    print('Количество посадочных мест:', place['SeatsCount']) 
    print('Социальные льготы:', place['SocialPrivileges'])
    print('Адрес:', place['Address']) 
    print('-------------------------------')

Всего найдено мест: 235
Название: Алло Пицца
Тип: кафе
Количество посадочных мест: 15
Социальные льготы: нет
Адрес: Российская Федерация, город Москва, внутригородская территория муниципальный округ Марьино, Новомарьинская улица, дом 14/15
-------------------------------
Название: Суши Wok
Тип: кафе
Количество посадочных мест: 11
Социальные льготы: нет
Адрес: город Москва, Братиславская улица, дом 15, корпус 1
-------------------------------
Название: Суши Wok
Тип: кафе
Количество посадочных мест: 12
Социальные льготы: нет
Адрес: Российская Федерация, город Москва, внутригородская территория муниципальный округ Марьино, Новомарьинская улица, дом 4
-------------------------------
Название: Суши Wok
Тип: кафе
Количество посадочных мест: 15
Социальные льготы: нет
Адрес: Российская Федерация, город Москва, внутригородская территория муниципальный округ Марьино, Новочеркасский бульвар, дом 51
-------------------------------
Название: Тануки
Тип: ресторан
Количество посадочных мест: 50
Социа

## Информация о заведении по названию
Напишем функцию, которая по названию заведения общественного питания будет выдавать всю имеющуюся информацию о нем.

In [32]:
def find_place(food_service_dict, name_inp):
    if name_inp in food_service_dict:
        places = food_service_dict[name_inp]
        number = len(food_service_dict[name_inp])
        for place in places:
            print('Общее количсество заведений:', number)
            print('Название:', name_inp)
            print('Тип заведения:', place['TypeObject'])
            print('Сетевое заведение:', place['IsNetObject'])
            print('Количество посадочных мест:', place['SeatsCount'])
            print('Административный округ:', place['AdmArea'])
            print('Район Москвы:', place['District'])
            print('Наличие социальных льгот:', place['SocialPrivileges'])
            print('Адрес:', place['Address'])
            print('Координаты в WGS84 (широта, долгота):', place['Latitude_WGS84'], place['Longitude_WGS84'])
            print('-------------------------------')  
    else: print('Заведение с таким названием не найдено')
    return find_place   

Например, можем найти информацию о кафе `Теремок`.

In [46]:
name_inp = 'Теремок' 
find_place(food_service_dict, name_inp)

Общее количсество заведений: 134
Название: Теремок
Тип заведения: кафе
Сетевое заведение: да
Количество посадочных мест: 20
Административный округ: Центральный административный округ
Район Москвы: Басманный район
Наличие социальных льгот: нет
Адрес: город Москва, Мясницкая улица, дом 30/1/2, строение 2
Координаты в WGS84 (широта, долгота): 55.7651407212443640 37.6389133980546760
-------------------------------
Общее количсество заведений: 134
Название: Теремок
Тип заведения: кафе
Сетевое заведение: да
Количество посадочных мест: 16
Административный округ: Центральный административный округ
Район Москвы: район Хамовники
Наличие социальных льгот: нет
Адрес: город Москва, Гоголевский бульвар, дом 3
Координаты в WGS84 (широта, долгота): 55.7450469999999000 37.6004670000000000
-------------------------------
Общее количсество заведений: 134
Название: Теремок
Тип заведения: предприятие быстрого обслуживания
Сетевое заведение: да
Количество посадочных мест: 34
Административный округ: Централь

<function __main__.find_place(food_service_dict, name_inp)>

Или, например, найти информацию о заведении `Брусника`

In [45]:
name_inp = 'Брусника' 
result = find_place(food_service_dict, name_inp)

Общее количсество заведений: 17
Название: Брусника
Тип заведения: кафе
Сетевое заведение: да
Количество посадочных мест: 10
Административный округ: Центральный административный округ
Район Москвы: район Арбат
Наличие социальных льгот: нет
Адрес: город Москва, переулок Сивцев Вражек, дом 6/2
Координаты в WGS84 (широта, долгота): 55.7473904905259940 37.5981275484399900
-------------------------------
Общее количсество заведений: 17
Название: Брусника
Тип заведения: кафе
Сетевое заведение: да
Количество посадочных мест: 40
Административный округ: Центральный административный округ
Район Москвы: Мещанский район
Наличие социальных льгот: нет
Адрес: Российская Федерация, город Москва, внутригородская территория муниципальный округ Мещанский, проспект Мира, дом 49
Координаты в WGS84 (широта, долгота): 55.7835133898564310 37.6338314726141970
-------------------------------
Общее количсество заведений: 17
Название: Брусника
Тип заведения: кафе
Сетевое заведение: да
Количество посадочных мест: 3

## Выводы
В итоге можно сказать, что я создала словарь, в котором храниться теперь информация о стационарных предприятиях общественного питания, расположенных в пределах города Москвы. Далее был проведен небольшой анализ этих данных.

Всего в базе содержится **205 505** точек общественного питания, **13 695** из которых обладают уникальным названием и не повторяются. 

##### **Больше всего** мест общественного питания располагается в следующих **административных округах**:
- Центральный административный округ: 5670 мест- Южный административный округ: 2488 мест
- Северный административный округ: 2032 места

##### А **меньше** всего в:
- Троицкий административный округ: 183 места
- Зеленоградский административный округ: 341 место
- Новомосковский административный округ: 984 места

##### Если говорить о **типах** предприятий, то **больше** всего оказалось:
- кафе: 7704 места
- предприятие быстрого обслуживани: 3799 мест
- ресторан: 2524 места
- кафетерий: 2227 мест
##### Также в рамках данной работы было написано две функции*:
1. Функция, которая позволяет найти все места, которые находятся в том или ином районе
2. Функция, которая выдает всю имеющуюся информацию о заведении при вводе названия

**функции воспринимают только те названия района или названия заведения, которые есть в изначальном датасете, то есть в том же регисте падаже и т.д.*)
